<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/multi_class_U_Net_Model_(11_march_2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [124]:
# multi-class u-net model

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [125]:
pip install rasterio


In [126]:
import rasterio

In [127]:
from sklearn.model_selection import train_test_split
import shutil

In [128]:
# Ensure the lists are sorted so that images and labels correspond to each other
planet_labels = sorted(glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/*.tif"))
planet_images = sorted(glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/*.tif"))

# Split the file paths into training and testing sets (80% train, 20% test)
train_images_paths, test_images_paths, train_labels_paths, test_labels_paths = train_test_split(
    planet_images, planet_labels, test_size=0.2, random_state=42)

In [129]:
# List of files to copy
# files_to_copy = ['path/to/source/file1.txt', 'path/to/source/file2.jpg', 'path/to/source/file3.pdf']
# Destination directory
destination = '/content/train_images'

# Ensure the destination directory exists
os.makedirs(destination, exist_ok=True)

# Loop through the list and copy each file
for file in train_images_paths:
    # Define the full destination path for the current file
    dest_file = os.path.join(destination, os.path.basename(file))
    # Copy the file
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')



# Destination directory
destination = '/content/train_labels'

# Ensure the destination directory exists
os.makedirs(destination, exist_ok=True)

# Loop through the list and copy each file
for file in train_labels_paths:
    # Define the full destination path for the current file
    dest_file = os.path.join(destination, os.path.basename(file))
    # Copy the file
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')

# Destination directory
destination = '/content/test_images'

# Ensure the destination directory exists
os.makedirs(destination, exist_ok=True)

# Loop through the list and copy each file
for file in test_images_paths:
    # Define the full destination path for the current file
    dest_file = os.path.join(destination, os.path.basename(file))
    # Copy the file
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')


# Destination directory
destination = '/content/test_labels'

# Ensure the destination directory exists
os.makedirs(destination, exist_ok=True)

# Loop through the list and copy each file
for file in test_labels_paths:
    # Define the full destination path for the current file
    dest_file = os.path.join(destination, os.path.basename(file))
    # Copy the file
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')

Streaming output truncated to the last 5000 lines.
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/311_planet_stack_utm4.tif to /content/train_images/311_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/2455_planet_stack_utm4.tif to /content/train_images/2455_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1592_planet_stack_utm4.tif to /content/train_images/1592_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/801_planet_stack_utm4.tif to /content/train_images/801_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1569_planet_stack_utm4.tif to /content/train_images/1569_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/650_planet_stack_utm4.tif to /content/train_images/650_planet_stac

In [130]:

test_images = sorted(glob.glob(f"/content/test_images/*.tif"))
test_labels = sorted(glob.glob(f"/content/test_labels/*.tif"))


train_images = sorted(glob.glob(f"/content/train_images/*.tif"))
train_labels = sorted(glob.glob(f"/content/train_labels/*.tif"))


In [131]:
#Resizing images, if needed
SIZE_X = 128
SIZE_Y = 128
n_classes= 13 #Number of classes for segmentation
batch_size = 64

In [132]:
# from tensorflow.keras.utils import Sequence, to_categorical
# import numpy as np
# import rasterio
# import cv2

# class DataGenerator(Sequence):
#     def __init__(self, image_files, label_files, img_height, img_width, batch_size, num_classes):
#         self.image_files = image_files
#         self.label_files = label_files
#         self.img_height = img_height
#         self.img_width = img_width
#         self.batch_size = batch_size
#         self.num_classes = num_classes


#     def __len__(self):
#         return int(np.ceil(len(self.image_files) / self.batch_size))

#     def __getitem__(self, index):
#         # Determine batch file paths
#         image_batch_files = self.image_files[index * self.batch_size : (index + 1) * self.batch_size]
#         label_batch_files = self.label_files[index * self.batch_size : (index + 1) * self.batch_size]
#         # Load and process images and labels
#         batch_images, batch_labels = self.load_images_and_labels(image_batch_files, label_batch_files)
#         return batch_images, batch_labels

#     def load_and_reshape_image(self, image_path, normalize=True):
#         with rasterio.open(image_path) as src:
#             image = src.read().transpose((1, 2, 0))
#             if image.shape[:2] != (self.img_height, self.img_width):
#                 image = cv2.resize(image, (self.img_width, self.img_height), interpolation=cv2.INTER_NEAREST)

#             if normalize:
#                 # Normalize the image data to 0-1 range
#                 image = image.astype(np.float32) / 255.0

#         return image


#     def load_images_and_labels(self, image_files, label_files):
#         images = []
#         labels = []

#         for image_file in image_files:
#             image = self.load_and_reshape_image(image_file)
#             images.append(image)

#         for label_file in label_files:
#             label = self.load_and_reshape_image(label_file, normalize=False)
#             label = label.astype(np.uint8)
#             label = to_categorical(label, num_classes=self.num_classes)
#             labels.append(label)

#         # images = [self.load_and_reshape_image(image_file) for image_file in image_files]
#         # labels = [self.load_and_reshape_image(label_file, normalize=False) for label_file in label_files]
#         # # Here, assuming labels are categorical and do not require normalization like images
#         # # For segmentation tasks, ensure labels are processed according to your task requirements
#         # labels = [to_categorical(label, num_classes=self.num_classes) for label in labels]
#         return np.array(images), np.array(labels)


In [133]:
from tensorflow.keras.utils import Sequence, to_categorical
import numpy as np
import rasterio
import cv2

class DataGenerator(Sequence):
    def __init__(self, image_files, label_files, img_height, img_width, batch_size, num_classes):
        self.image_files = image_files
        self.label_files = label_files
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size
        self.num_classes = num_classes

    def __len__(self):
        return int(np.ceil(len(self.image_files) / self.batch_size))

    def __getitem__(self, index):
        image_batch_files = self.image_files[index * self.batch_size : (index + 1) * self.batch_size]
        label_batch_files = self.label_files[index * self.batch_size : (index + 1) * self.batch_size]

        batch_images, batch_labels = self.load_images_and_labels(image_batch_files, label_batch_files)
        return batch_images, batch_labels

    def load_and_reshape_image(self, image_path):
        with rasterio.open(image_path) as src:
            image = src.read().transpose((1, 2, 0))
            if image.shape[:2] != (self.img_height, self.img_width):
                image = cv2.resize(image, (self.img_width, self.img_height), interpolation=cv2.INTER_NEAREST)
        return image

    # def z_score_normalization(self, image):
    #     # epsilon = 1e-7
    #     # mean = np.mean(image, axis=(0, 1), keepdims=True)
    #     # std = np.std(image, axis=(0, 1), keepdims=True)
    #     # std[std < 0] = 0
    #     # normalized_image = (image - mean) / (std + epsilon)

    #     min = np.min(image, axis=(0, 1), keepdims=True)
    #     max = np.max(image, axis=(0, 1), keepdims=True)
    #     normalized_image = (image - min) / (max - min)


    #     return normalized_image

    def z_score_normalization(self, image):
        image = image.astype(np.float32)  # Ensure the image is in float format to handle division and large ranges

        min_val = np.min(image, axis=(0, 1), keepdims=True)
        max_val = np.max(image, axis=(0, 1), keepdims=True)
        range_val = max_val - min_val

        # Avoid division by zero by checking if range is 0
        if np.any(range_val == 0):
            normalized_image = np.zeros(image.shape, dtype=np.float32)
        else:
            normalized_image = (image - min_val) / range_val

        return normalized_image




# data = np.random.normal(loc=0, scale=1, size=(96108, 7, 7))
# data_min = np.min(data, axis=(1,2), keepdims=True)
# data_max = np.max(data, axis=(1,2), keepdims=True)

# scaled_data = (data - data_min) / (data_max - data_min)

    def load_images_and_labels(self, image_files, label_files):
        images = [self.load_and_reshape_image(image_file) for image_file in image_files]
        labels = [self.load_and_reshape_image(label_file) for label_file in label_files]

        # Apply Z-score normalization to images
        normalized_images = [self.z_score_normalization(image) for image in images]

        # Process labels if necessary (for example, converting to categorical)
        processed_labels = [to_categorical(label, num_classes=self.num_classes) for label in labels]

        return np.array(normalized_images), np.array(processed_labels)


test_images = sorted(glob.glob(f"/content/test_images/*.tif"))
test_labels = sorted(glob.glob(f"/content/test_labels/*.tif"))


train_images = sorted(glob.glob(f"/content/train_images/*.tif"))
train_labels = sorted(glob.glob(f"/content/train_labels/*.tif"))

In [134]:
# train_data_generator = DataGenerator(train_images_paths, train_labels_paths, SIZE_X, SIZE_Y, batch_size, n_classes)
train_data_generator = DataGenerator(train_images, train_labels, SIZE_X, SIZE_Y, batch_size, n_classes)
test_data_generator = DataGenerator(test_images, test_labels, SIZE_X, SIZE_Y, batch_size, n_classes)

In [135]:
batch_images, batch_labels = train_data_generator.__getitem__(0)

In [136]:
batch_images.shape

(64, 128, 128, 13)

In [137]:
batch_labels.shape

(64, 128, 128, 13)

In [138]:
batch_images

array([[[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 6.92139745e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 7.16173828e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 7.16173828e-01],
         ...,
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 9.30202603e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 9.30202603e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 9.30202603e-01]],

        [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 6.88653946e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 7.13692844e-01],
        

In [139]:
batch_labels

array([[[[0., 0., 0., ..., 0., 1., 0.],
         [0., 0., 0., ..., 0., 1., 0.],
         [0., 0., 0., ..., 0., 1., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 1., 0.],
         [0., 0., 0., ..., 0., 1., 0.],
         [0., 0., 0., ..., 0., 1., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 1., 0.],
         [0., 0., 0., ..., 0., 1., 0.],
         [0., 0., 0., ..., 0., 1., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 1., 0.],
         [0., 0., 0., ..., 0., 1., 0.],
         [0., 0., 0., ..., 0., 1., 0.],
         ...,
         [0., 0., 0., ..., 0., 1., 0.],
         [0., 0., 0., ..., 0., 1., 0.],
    

In [140]:
# prompt: print how many files are in y_test
print('Number of files in train labels:', len(batch_images))
print('Number of files in test labels:', len(batch_images))
batch_images.shape

Number of files in train labels: 64
Number of files in test labels: 64


(64, 128, 128, 13)

In [141]:
# prompt: print how many files are in y_test
print('Number of files in train labels:', len(batch_labels))
print('Number of files in test labels:', len(batch_labels))
batch_labels.shape

Number of files in train labels: 64
Number of files in test labels: 64


(64, 128, 128, 13)

In [142]:
# # Split the file paths into training and testing sets (80% train, 20% test)
# train_images_paths, test_images_paths, train_labels_paths, test_labels_paths = train_test_split(
#     planet_images, planet_labels, test_size=0.2, random_state=42)

In [143]:
from keras.utils import to_categorical
import keras
# import TenserFlow classes and functions
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization

In [144]:
# train_labels_cat = to_categorical(train_labels_paths, num_classes=n_classes)
# y_train_cat = train_labels_cat.reshape((train_labels_paths.shape[0], train_labels_paths.shape[1], train_labels_paths.shape[2], n_classes))
# test_labels_cat = to_categorical(test_labels_paths, num_classes=n_classes)
# y_test_cat = test_labels_cat.reshape((test_labels_paths.shape[0], test_labels_paths.shape[1], test_labels_paths.shape[2], n_classes))

In [145]:
def multi_unet_model(n_classes=13, IMG_HEIGHT=128, IMG_WIDTH=128, IMG_CHANNELS=13):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    #Expansive path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(n_classes, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    #NOTE: Compile the model in the main program to make it easy to test with various loss functions
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    #model.summary()

    return model

In [146]:
model = multi_unet_model()

In [147]:
print(model.summary()) # 1,942,732

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 128, 128, 13)]       0         []                            
                                                                                                  
 conv2d_95 (Conv2D)          (None, 128, 128, 16)         1888      ['input_6[0][0]']             
                                                                                                  
 dropout_45 (Dropout)        (None, 128, 128, 16)         0         ['conv2d_95[0][0]']           
                                                                                                  
 conv2d_96 (Conv2D)          (None, 128, 128, 16)         2320      ['dropout_45[0][0]']          
                                                                                            

In [154]:
from tensorflow.keras.optimizers import Adam

In [156]:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', "categorical_accuracy"])
# callbacks = [keras.callbacks.ModelCheckpoint(filepath='model_100.h5', save_best_only=True)]

model.compile(optimizer= Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy', "categorical_accuracy"])
callbacks = [keras.callbacks.ModelCheckpoint(filepath='model_100_lr001.h5', save_best_only=True)]


In [157]:
model.fit(train_data_generator, validation_data= test_data_generator, epochs=100, callbacks=callbacks,shuffle=True)

Epoch 1/100
36/36 [==============================] - 231s 6s/step - loss: 1.3389 - accuracy: 0.5439 - categorical_accuracy: 0.5439 - val_loss: 1.4289 - val_accuracy: 0.5286 - val_categorical_accuracy: 0.5286
Epoch 2/100
36/36 [==============================] - 228s 6s/step - loss: 1.3015 - accuracy: 0.5539 - categorical_accuracy: 0.5539 - val_loss: 1.4250 - val_accuracy: 0.5239 - val_categorical_accuracy: 0.5239
Epoch 3/100
36/36 [==============================] - 235s 7s/step - loss: 1.3665 - accuracy: 0.5519 - categorical_accuracy: 0.5519 - val_loss: 1.3965 - val_accuracy: 0.5296 - val_categorical_accuracy: 0.5296
Epoch 4/100
36/36 [==============================] - 236s 7s/step - loss: 1.3486 - accuracy: 0.5513 - categorical_accuracy: 0.5513 - val_loss: 1.4764 - val_accuracy: 0.4998 - val_categorical_accuracy: 0.4998
Epoch 5/100
36/36 [==============================] - 236s 7s/step - loss: 1.3105 - accuracy: 0.5481 - categorical_accuracy: 0.5481 - val_loss: 1.4191 - val_accuracy: 0.

KeyboardInterrupt: 